In [1]:
from urllib.request import urlopen
from IPython.display import SVG
import matplotlib.pyplot as plt
from rdkit import Chem
from tqdm import tqdm
import pandas as pd
import xlsxwriter
import argparse
import pickle
import numpy as np
import json

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import visualizer as visualizer
import utils as utils
import handle_network as hn
import fragmentation_py as fragmentation_py
import library_downloader as library_downloader
import SiteLocator as modSite

In [2]:
libraries = {
    "GNPS-MSMLS": "https://external.gnps2.org/gnpslibrary/GNPS-MSMLS.json",
    "GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE": "https://external.gnps2.org/gnpslibrary/GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE.json",
    "GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE": "https://external.gnps2.org/gnpslibrary/GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE.json",
    "MIADB": "https://external.gnps2.org/gnpslibrary/MIADB.json",
    "BERKELEY-LAB": "https://external.gnps2.org/gnpslibrary/BERKELEY-LAB.json"
    # "GNPS-LIBRARY": "https://gnps-external.ucsd.edu/gnpslibrary/GNPS-LIBRARY.json"
}

In [3]:
library ="BERKELEY-LAB"
if not os.path.exists( os.path.join("../data/libraries", library)):
    url = "https://gnps-external.ucsd.edu/gnpslibrary/" + library + ".json"
    location = "../data/libraries/" + library + "/"
    library_downloader.download(url, location, 0.5, 0.1)

with open(os.path.join("../data/libraries", library, "data_dict_filtered.pkl"), "rb") as f:
    data_dict_filtered = pickle.load(f)

# load matches
with open(os.path.join("../data/libraries", library, "matches.pkl"), "rb") as f:
    matches = pickle.load(f)

# load cachedStructures_filtered
with open(os.path.join("../data/libraries", library, "cachedStructures.pkl"), "rb") as f:
    cachedStructures_filtered = pickle.load(f)

In [4]:
print (matches[1].pop())

('CCMSLIB00010113508', 'CCMSLIB00010121997')


In [5]:
print (data_dict_filtered['CCMSLIB00010105110']['Precursor_MZ'], data_dict_filtered['CCMSLIB00010114866']['Precursor_MZ'])

253.051 237.056


In [6]:
## create cashe helpers
helpers = dict()
for match in matches[1]:
    if match[0] not in helpers:
        helpers[match[0]] = []
    helpers[match[0]].append(match[1])

print(len(helpers))

6778


In [7]:
colums = ["mol1ID", "mol2ID", "mol1smile", "mol2smile", "delta_mass",
        "#_matched_peaks", "#_shifted_peaks", "#_unshifted_peaks", 
        "Closest_Max_Atom_Distance", "Count_Max", "Is_Max", "cosine", 
        "score", "best_score", "random_guess", "random_prob", "url"]




In [8]:
print (data_dict_filtered['CCMSLIB00010105110'].keys())
print(type(json.loads(data_dict_filtered['CCMSLIB00010105110']['peaks_json'])[0][1]))

dict_keys(['spectrum_id', 'source_file', 'task', 'scan', 'ms_level', 'library_membership', 'spectrum_status', 'peaks_json', 'splash', 'submit_user', 'Compound_Name', 'Ion_Source', 'Compound_Source', 'Instrument', 'PI', 'Data_Collector', 'Adduct', 'Scan', 'Precursor_MZ', 'ExactMass', 'Charge', 'CAS_Number', 'Pubmed_ID', 'Smiles', 'INCHI', 'INCHI_AUX', 'Library_Class', 'SpectrumID', 'Ion_Mode', 'create_time', 'task_id', 'user_id', 'InChIKey_smiles', 'InChIKey_inchi', 'Formula_smiles', 'Formula_inchi', 'url', 'annotation_history'])
<class 'float'>


In [9]:
m0, m1 = matches[1].pop()
molMol = cachedStructures_filtered[m1]
modifMol = cachedStructures_filtered[m0]
molUsi = hn.generate_usi(m1, library)
modifUsi = hn.generate_usi(m0, library)
molSmiles = data_dict_filtered[m1]['Smiles']
modifSmiles = data_dict_filtered[m0]['Smiles']
site = modSite.SiteLocator(data_dict_filtered[m1], data_dict_filtered[m0], molMol)

{'adduct': 'M+H', 'filter_peaks_method': 'top_k', 'filter_peaks_variable': 50, 'mz_tolerance': 0.05, 'ppm': 1.01, 'min_score_ratio': 0.5, 'distance_decay': 0.1}
[[78.001022, 2845.0], [82.384644, 3093.0], [82.544777, 2218.0], [98.922401, 2464.0], [115.963432, 2368.0], [122.392197, 2604.0], [130.065186, 17522.0], [146.059952, 15551.0], [157.947876, 3406.0], [158.060211, 32395.0], [231.112534, 30293.0], [243.112549, 296748.0], [271.10733, 1036091.0], [284.137878, 6126.0]]


In [25]:
df = pd.DataFrame(columns=colums)
count = 0
for match in tqdm(matches[1]):
    if count > 100:
        break
    try:
        
        m0, m1 = match
        if data_dict_filtered[m0]['Adduct'] != data_dict_filtered[m1]['Adduct'] or data_dict_filtered[m0]['Adduct'] != "M+H":
            continue
        molMol = cachedStructures_filtered[m1]
        modifMol = cachedStructures_filtered[m0]
        molUsi = hn.generate_usi(m1, library)
        modifUsi = hn.generate_usi(m0, library)
        molSmiles = data_dict_filtered[m1]['Smiles']
        modifSmiles = data_dict_filtered[m0]['Smiles']
        site = modSite.SiteLocator(data_dict_filtered[m1], data_dict_filtered[m0], molSmiles)
        modifLoc = utils.calculateModificationSites(modifMol, molMol, False)
        peak_presence_only = False
        combine = True
        # calculate score
        res = site.accuracy_score(modifLoc[0], peak_presence_only=peak_presence_only, combine=combine, return_all=True)

        # generate random probability array 1-hot
        prob = np.zeros(site.molMol.GetNumAtoms())
        randInt = np.random.randint(0, site.molMol.GetNumAtoms())
        prob[randInt] = 1
        res2 = site.tempScore(modifLoc[0], prob, True)

        # generate random probability array distribution
        prb = np.random.rand(site.molMol.GetNumAtoms())
        prb = prb / prb.sum()
        res3 = site.tempScore(modifLoc[0], prb, True)

        # get max score
        maxScore = site.get_max_possible_score(modifLoc[0], peak_presence_only=peak_presence_only, combine=combine)
        
    # "mol1ID", "mol2ID", "mol1smile", "mol2smile", "delta_mass",
    #         "#_matched_peaks", "#_shifted_peaks", "#_unshifted_peaks", 
    #         "Closest_Max_Atom_Distance", "Count_Max", "Is_Max", "cosine", 
    #         "score", "best_score", "random_guess", "random_prob", "url"

        df = pd.concat([df, pd.DataFrame.from_records([{"mol1ID": molUsi, "mol2ID": modifUsi, "mol1smile": molSmiles, "mol2smile": data_dict_filtered[m0]['Smiles'], 
                                                        "delta_mass": abs(float(data_dict_filtered[m0]['Precursor_MZ']) - float(data_dict_filtered[m1]['Precursor_MZ'])),
                                                        "#_matched_peaks": len(site.matchedPeaks), "#_shifted_peaks": len(site.shifted), "#_unshifted_peaks": len(site.unshifted),
                                                        "Closest_Max_Atom_Distance": res['closestMaxAtomDistance'], "Count_Max": res['count'], "Is_Max": res['isMax'], "cosine":site.cosine, 
                                                        "score": res['score'], "best_score": maxScore, "random_guess":res2['score'], "random_prob":res3['score'], 
                                                        "url":visualizer.make_url("http://reza.cs.ucr.edu/", molUsi, modifUsi, molSmiles, modifSmiles, args=None) }])], ignore_index=True)
        count += 1
    except:
        # print stack trace
        # import traceback
        # traceback.print_exc()
        pass


  0%|          | 0/27814 [00:00<?, ?it/s]

{'adduct': 'M+H', 'filter_peaks_method': 'top_k', 'filter_peaks_variable': 50, 'mz_tolerance': 0.05, 'ppm': 1.01, 'min_score_ratio': 0.5, 'distance_decay': 0.1}
[[65.039177, 24347.0], [82.393547, 3083.0], [85.268768, 3123.0], [91.054413, 3355371.0], [115.113907, 3687.0], [123.31913, 2949.0], [146.058929, 3839.0], [167.440704, 3226.0], [174.092636, 3187.0], [183.101181, 2894.0], [188.07045, 40210.0], [202.086182, 38907.0], [262.190186, 3757.0], [280.132996, 3297619.0]]
{'adduct': 'M+H', 'filter_peaks_method': 'top_k', 'filter_peaks_variable': 50, 'mz_tolerance': 0.05, 'ppm': 1.01, 'min_score_ratio': 0.5, 'distance_decay': 0.1}
[[53.038921, 4090.0], [55.018261, 5393.0], [57.03442, 2462.0], [65.038986, 18198.0], [79.05455, 6835.0], [84.960037, 26482.0], [89.039093, 4122.0], [91.054367, 218576.0], [92.375977, 3493.0], [103.053963, 13694.0], [105.033188, 4456.0], [105.069847, 3836.0], [107.04892, 14861.0], [115.054153, 3748.0], [118.04129, 19884.0], [119.01255, 28974.0], [119.049149, 133963

  0%|          | 13/27814 [00:00<14:29, 31.96it/s]

{'adduct': 'M+H', 'filter_peaks_method': 'top_k', 'filter_peaks_variable': 50, 'mz_tolerance': 0.05, 'ppm': 1.01, 'min_score_ratio': 0.5, 'distance_decay': 0.1}
[[53.002739, 10768.0], [56.219601, 10511.0], [63.86161, 11151.0], [65.038986, 42848.0], [66.740646, 10660.0], [67.018257, 894382.0], [68.997498, 505966.0], [69.033836, 18342.0], [79.209641, 10321.0], [81.034012, 26202.0], [83.013077, 11728.0], [88.964897, 9779.0], [90.176567, 8972.0], [91.01799, 10656.0], [91.054359, 638594.0], [95.049522, 20987.0], [97.028236, 50280.0], [104.284142, 13776.0], [109.028542, 25437.0], [109.318283, 11670.0], [111.007927, 79095.0], [112.026367, 10533.0], [119.049187, 3056672.0], [121.028389, 843583.0], [125.023193, 54233.0], [127.038757, 37325.0], [129.018066, 248785.0], [131.049484, 38886.0], [141.070236, 108479.0], [143.034332, 16912.0], [145.028397, 799800.0], [145.064423, 23471.0], [149.023224, 18516.0], [153.018219, 6555352.0], [155.048508, 17626.0], [157.064178, 44445.0], [161.060638, 14973.0

  0%|          | 17/27814 [00:00<29:41, 15.61it/s]/home/user/Substructure_Assignment/sharable/SiteLocator.py:214: RuntimeWarning:

invalid value encountered in divide



...

  1%|          | 239/27814 [00:20<1:44:29,  4.40it/s]

{'adduct': 'M+H', 'filter_peaks_method': 'top_k', 'filter_peaks_variable': 50, 'mz_tolerance': 0.05, 'ppm': 1.01, 'min_score_ratio': 0.5, 'distance_decay': 0.1}
[[53.039261, 6552.0], [55.71534, 2783.0], [56.781601, 3024.0], [66.705482, 2966.0], [66.953743, 2959.0], [79.054359, 5751.0], [81.463966, 2908.0], [85.028763, 5466.0], [88.602722, 3002.0], [107.049103, 7174699.0], [121.064919, 18997.0], [127.038834, 100002.0], [140.954483, 3522.0], [149.046051, 6284.0], [149.059677, 170381.0], [151.038757, 53704.0], [167.071198, 3467.0], [169.049469, 1479185.0], [181.049072, 22732.0], [211.201172, 3501.0], [232.053802, 3129.0], [234.988602, 3828.0], [257.080902, 15710.0], [275.091034, 658729.0]]
{'adduct': 'M+H', 'filter_peaks_method': 'top_k', 'filter_peaks_variable': 50, 'mz_tolerance': 0.05, 'ppm': 1.01, 'min_score_ratio': 0.5, 'distance_decay': 0.1}
[[51.02372, 5421.0], [55.94706, 3797.0], [63.23455, 3206.0], [65.039009, 66837.0], [65.450974, 3968.0], [74.727081, 3493.0], [77.034813, 4245.0

  1%|          | 251/27814 [00:20<39:50, 11.53it/s]  

{'adduct': 'M+H', 'filter_peaks_method': 'top_k', 'filter_peaks_variable': 50, 'mz_tolerance': 0.05, 'ppm': 1.01, 'min_score_ratio': 0.5, 'distance_decay': 0.1}
[[51.023621, 5607.0], [51.817719, 2522.0], [53.039162, 20643.0], [63.06749, 3398.0], [77.038788, 6915.0], [84.959396, 5088.0], [85.877922, 3316.0], [92.049202, 24989.0], [95.04892, 6858.0], [102.046021, 11831.0], [103.0541, 1980999.0], [105.069862, 26440.0], [107.060226, 131416.0], [108.067909, 5303.0], [109.075943, 314696.0], [110.060043, 16188.0], [121.064812, 4993.0], [131.049011, 3607053.0], [133.075867, 28661.0], [134.060165, 3648.0], [144.080536, 14858.0], [149.070709, 16153.0], [151.086349, 385003.0], [161.070572, 42360.0], [177.064926, 4234.0], [180.080139, 13328.0], [203.081284, 64563.0], [221.106674, 7083.0], [239.11792, 38793.0], [263.117218, 12992.0], [281.127808, 2322347.0]]
{'adduct': 'M+H', 'filter_peaks_method': 'top_k', 'filter_peaks_variable': 50, 'mz_tolerance': 0.05, 'ppm': 1.01, 'min_score_ratio': 0.5, 'dis

  1%|          | 252/27814 [00:21<38:41, 11.87it/s]


In [30]:
import plotly.graph_objects as go


# Bin the data by delta_mass into discrete bins
df['delta_mass_bin'] = pd.cut(df['delta_mass'], bins=10)

# Calculate the average for each bin
df_grouped = df.groupby('delta_mass_bin').mean().reset_index()


# Create the plot
fig = go.Figure()

# Add scatter plot for each column
columns = ["score", "best_score", "random_guess", "random_prob"]
for col in columns:
    fig.add_trace(go.Scatter(x=df_grouped["delta_mass"], y=df_grouped[col], mode='lines+markers', name=col))

# Customize the plot
fig.update_layout(
    title="Comparison of Scores",
    xaxis_title="delta_mass",
    yaxis_title="Score",
    showlegend=True,
)

# Show the plot
fig.show()


/tmp/ipykernel_2522607/1351017751.py:8: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [ ]:
df2 = pd.DataFrame(columns=colums)
count = 0
for match in tqdm(matches[1]):
    if count > 100:
        break
    try:
        m0, m1 = match
        if data_dict_filtered[m0]['Adduct'] != data_dict_filtered[m1]['Adduct'] or data_dict_filtered[m0]['Adduct'] != "M+H":
            continue
        molMol = cachedStructures_filtered[m1]
        modifMol = cachedStructures_filtered[m0]
        molUsi = hn.generate_usi(m1, library)
        modifUsi = hn.generate_usi(m0, library)
        molSmiles = data_dict_filtered[m1]['Smiles']
        modifSmiles = data_dict_filtered[m0]['Smiles']
        site = modSite.SiteLocator(data_dict_filtered[m1], data_dict_filtered[m0], molSmiles)
        for helper in helpers[m1]:
            if helper != m0:
                try:
                    site.helper_molecule(data_dict_filtered[helper], data_dict_filtered[helper]['Smiles'])
                except:
                    pass
        modifLoc = utils.calculateModificationSites(modifMol, molMol, False)
        peak_presence_only = False
        combine = True
        # calculate score
        res = site.accuracy_score(modifLoc[0], peak_presence_only=peak_presence_only, combine=combine, return_all=True)

        # generate random probability array 1-hot
        prob = np.zeros(site.molMol.GetNumAtoms())
        randInt = np.random.randint(0, site.molMol.GetNumAtoms())
        prob[randInt] = 1
        res2 = site.tempScore(modifLoc[0], prob, True)

        # generate random probability array distribution
        prb = np.random.rand(site.molMol.GetNumAtoms())
        prb = prb / prb.sum()
        res3 = site.tempScore(modifLoc[0], prb, True)

        # get max score
        maxScore = site.get_max_possible_score(modifLoc[0], peak_presence_only=peak_presence_only, combine=combine)
        
    # "mol1ID", "mol2ID", "mol1smile", "mol2smile", "delta_mass",
    #         "#_matched_peaks", "#_shifted_peaks", "#_unshifted_peaks", 
    #         "Closest_Max_Atom_Distance", "Count_Max", "Is_Max", "cosine", 
    #         "score", "best_score", "random_guess", "random_prob", "url"

        df2 = pd.concat([df, pd.DataFrame.from_records([{"mol1ID": molUsi, "mol2ID": modifUsi, "mol1smile": molSmiles, "mol2smile": data_dict_filtered[m0]['Smiles'], 
                                                        "delta_mass": abs(float(data_dict_filtered[m0]['Precursor_MZ']) - float(data_dict_filtered[m1]['Precursor_MZ'])),
                                                        "#_matched_peaks": len(site.matchedPeaks), "#_shifted_peaks": len(site.shifted), "#_unshifted_peaks": len(site.unshifted),
                                                        "Closest_Max_Atom_Distance": res['closestMaxAtomDistance'], "Count_Max": res['count'], "Is_Max": res['isMax'], "cosine":site.cosine, 
                                                        "score": res['score'], "best_score": maxScore, "random_guess":res2['score'], "random_prob":res3['score'], 
                                                        "url":visualizer.make_url("http://reza.cs.ucr.edu/", molUsi, modifUsi, molSmiles, modifSmiles, args=None) }])], ignore_index=True)
        count += 1
    except:
        # print stack trace
        # import traceback
        # traceback.print_exc()
        pass


In [29]:
# Bin the data by delta_mass into discrete bins
df2['delta_mass_bin'] = pd.cut(df2['delta_mass'], bins=10)

# Calculate the average for each bin
df_grouped2 = df2.groupby('delta_mass_bin').mean().reset_index()


# Create the plot
fig = go.Figure()

# Add scatter plot for each column
columns = ["score", "best_score", "random_guess", "random_prob"]
for col in columns:
    fig.add_trace(go.Scatter(x=df_grouped2["delta_mass"], y=df_grouped2[col], mode='lines+markers', name=col))

# Customize the plot
fig.update_layout(
    title="Comparison of Scores",
    xaxis_title="delta_mass",
    yaxis_title="Score",
    showlegend=True,
)

# Show the plot
fig.show()


/tmp/ipykernel_2522607/4193020768.py:5: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

